## Eigenfaces
Another great use of eigenvalues and eigenvectors can be **feature detection**. In one algorithm, called **eigenfaces**, if you receive many images of faces, you can see "what makes a face". The principal characteristics of a face can be extracted using something similar to PCA.

You can see more info about the topic [at Wikipedia](https://en.wikipedia.org/wiki/Eigenface).

### Algorithm


#### Step 1 - Input dataset
The first step in the Eigenfaces algorithm is to input a dataset of N face images:

<figure>
<img src="img/eigenfaces/eigenfaces_1.png" >
<figcaption style="font-size: 14px; font-style: italic;"><b>Figure 1</b>: A sample of faces dataset.</figcaption>
</figure>

For face recognition to be successful (and somewhat robust), we should ensure we have multiple images per person we want to recognize.


#### Step 2 - Image representation
Let’s now consider an image containing a face:

<figure>
<img src="img/eigenfaces/eigenfaces_2.png" >
<figcaption style="font-size: 14px; font-style: italic; width: 400px;"><b>Figure 2</b>: Each face in our image is considered to be a KxK image (although non-square face images will certainly work as well, we’ll just use square images to make this lesson a bit easier to explain).</figcaption>
</figure>

When applying Eigenfaces, each face is represented as a grayscale, $K×K$ bitmap of pixels (images do not have to be square, but for the sake of this example, it’s easier to explain if we assume square images).


#### Step 3 - Flatten image matrix into vector
In order to apply the Eigenfaces algorithm, we need to form a single vector from the image. This is accomplished by **“flattening”** each image into a $K^{2}$-dim vector:

<figure>
<img src="img/eigenfaces/eigenfaces_3.png" >
<figcaption style="font-size: 14px; font-style: italic;"><b>Figure 3</b>: We can take our K×K image and then flatten it into a single “feature vector” of raw pixel intensities.</figcaption>
</figure>

Again, all we have done here is taken a K×K image and concatenated all of the rows together, forming a single, long $K^{2}$ list of grayscale pixel intensities.


#### Step 4 - Convert the dataset into a matrix of vectors
After each image in the dataset has been flattened, we form a matrix of flattened images like this, where $Z$ is the total number of images in our dataset and $K^2$ is the dimension of each image vector in the set:

<figure>
<img src="img/eigenfaces/eigenfaces_4.png" >
<figcaption style="font-size: 14px; font-style: italic; width: 480px;"><b>Figure 4</b>: Our entire faces dataset represented as a matrix with Z rows, the number of images, and K-squared columns, the flattened image representation.</figcaption>
</figure>

**Our entire dataset is now contained in a single matrix, $M$.**


#### Step 5 - Apply PCA

Given this matrix $M$, we are now ready to apply **Principal Component Analysis (PCA)**, the cornerstone of the Eigenfaces algorithm.

A complete review associated with the linear algebra underlying PCA is outside the scope of this article, but the general outline of the algorithm follows:

1. Compute the mean $\mu_{i}$ of each column in the matrix, giving us the average pixel intensity value for every $(x, y)$-coordinate in the image dataset.

2. Subtract the $\mu_{i}$ from each column $c_{i}$ — this is called **mean centering** the data and is a required step when performing PCA.

3. Now that our matrix $M$ has been **mean centered**, compute the **covariance matrix**.

4. Perform an **eigenvalue decomposition** on the covariance matrix to get the eigenvalues $\lambda_{i}$ and eigenvectors $\mathbf{X_{i}}$.

5. Sort $\mathbf{X_{i}}$ by $|\lambda_{i}|$, largest to smallest.

6. Take the top $N$ eigenvectors with the largest corresponding eigenvalue magnitude.

7. Transform the input data by projecting (i.e., taking the dot product) it onto the space created by the top $N$ eigenvectors — **these eigenvectors are called our eigenfaces**.

Again, a complete review on manually computing the covariance matrix and performing an eigenvalue decomposition is outside the scope of this article. For a more detailed review, please see the previous articles.

### Explanation
However, before we perform actual face identification using the Eigenfaces algorithm, let’s actually discuss these eigenface representations:

<figure>
<img src="img/eigenfaces/eigenfaces_5.png" >
<figcaption style="font-size: 14px; font-style: italic; width: 400px;"><b>Figure 5</b>: After applying an eigenvalue decomposition of the matrix, M, we are left with a matrix V, containing N rows (our eigenvectors) each of dimensionality K^2.</figcaption>
</figure>

Each row in the matrix above is an eigenface with $K^{2}$ entries — exactly like our original image.

What does this mean? Well, since each of these eigenface representations is actually a K^{2} vector, we can reshape it into a K×K bitmap:

<figure>
<img src="img/eigenfaces/eigenfaces_6.png" >
<figcaption style="font-size: 14px; font-style: italic; width: 600px;"><b>Figure 6</b>: Left: Mean face image. Right: The top 16-eigenface representations. Light regions demonstrate high variation in the dataset whereas dark areas suggest little variation.</figcaption>
</figure>

The image on the left is simply the average of all faces in our dataset, while the figures on the right show the most prominent deviations from the mean in our face dataset.

This can be thought of as a visualization of the dimension in which people’s faces vary the most. Lighter regions correspond to higher variation, where darker regions correspond to little to no variation. Here, we can see that our eigenface representation captures considerable variance in the eyes, hair, nose, lips, and cheek structure.

Now that we understand how the Eigenfaces representation is constructed, let’s move on to learning how we can actually identify faces using Eigenfaces.


### Identify faces using Eigenfaces
Given our eigenface vectors, we can represent a new face by taking the dot product between the (flattened) input face image and the $N$ eigenfaces. This allows us to represent each face as a **linear combination of principal components**:

Query Face = $36\%$ of Eigenface #1 $+ -8\%$ of Eigenface #2 $+ … + 21\%$ of Eigenface $N$

To perform the actual face identification, Sirovich and Kirby proposed taking the **Euclidean distance** between projected eigenface representations — this is, in essence, a $k-NN$ classifier:

<figure>
<img src="img/eigenfaces/eigenfaces_7.png" >
<figcaption style="font-size: 14px; font-style: italic; width: 600px;"><b>Figure 7</b>: Computing the Euclidean distance (denoted as function, d) between face representations. <b>The smaller the distance, the more similar the faces are considered to be.</b></figcaption>
</figure>

The smaller the Euclidean distance (denoted as the function, d), the more “similar” the two faces are — the overall identification is found by taking the label associated with the face with the smallest Euclidean distance.

For example, in **Figure 7** the top image pair has a distance of `0` because the two faces are identical (i.e., the same image).

The middle image pair has a distance of `0.07` — while the images are different they contain the same face.

The third image pair has a much larger distance (`9.81`), indicating that the two faces presented to the Eigenfaces algorithm are not the same person.

In practice, we often don’t rely on a simple k-NN algorithm for identification. Accuracy can be increased by using more advanced machine learning algorithms, such as Support Vector Machines (SVMs), Random Forests, etc.

## References
* [OpenCV Eigenfaces for Face Recognition](https://pyimagesearch.com/2021/05/10/opencv-eigenfaces-for-face-recognition/)